In [47]:
import pandas as pd
import numpy as np
import joblib




In [9]:
bdf = pd.read_csv('bureau.csv')

In [10]:
bdf.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [11]:
bdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int64  
 1   SK_ID_BUREAU            int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int64  
 5   CREDIT_DAY_OVERDUE      int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int64  
 16  AMT_ANNUITY             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB


In [12]:
obj_cols = bdf.select_dtypes(include='object').columns.tolist()
obj_cols


['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']

In [13]:
bdf.isnull().sum().sort_values(ascending=False).head(20)


AMT_ANNUITY               1226791
AMT_CREDIT_MAX_OVERDUE    1124488
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_DEBT        257669
DAYS_CREDIT_ENDDATE        105553
AMT_CREDIT_SUM                 13
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
SK_ID_BUREAU                    0
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
SK_ID_CURR                      0
dtype: int64

In bureau data:

If a loan has no DEBT → debt column is missing

If a loan is closed → end date missing

If loan has no credit limit → limit missing

If loan did not have overdue → overdue missing

Missing ≠ bad data
Missing = meaningful information

In [14]:
bdf = bdf.fillna(0)


In [ ]:
baldf=pd.read_csv('bureau_balance.csv')

In [21]:
baldf.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [18]:
baldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   SK_ID_BUREAU    int64 
 1   MONTHS_BALANCE  int64 
 2   STATUS          object
dtypes: int64(2), object(1)
memory usage: 624.8+ MB


In [27]:
baldf.isnull().sum().sort_values(ascending=False).head(20)



SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [29]:
baldf["STATUS"].value_counts()

STATUS
C    13646993
0     7499507
X     5810482
1      242347
5       62406
2       23419
3        8924
4        5847
Name: count, dtype: int64

Count number of months recorded for each loan


In [31]:
bb_agg = baldf.groupby("SK_ID_BUREAU")["MONTHS_BALANCE"].count().reset_index()
bb_agg.rename(columns={"MONTHS_BALANCE": "BB_MONTHS_COUNT"}, inplace=True)


Count overdue months (STATUS 1–5)

In [32]:
bb_dummies = pd.get_dummies(baldf["STATUS"], prefix="BB_STATUS")
baldf = pd.concat([baldf, bb_dummies], axis=1)


In [33]:
bb_status_agg = baldf.groupby("SK_ID_BUREAU")[[
    "BB_STATUS_0","BB_STATUS_1","BB_STATUS_2","BB_STATUS_3","BB_STATUS_4","BB_STATUS_5","BB_STATUS_C","BB_STATUS_X"
]].sum().reset_index()


Merge these aggregated features together

In [34]:
bb_final = bb_agg.merge(bb_status_agg, on="SK_ID_BUREAU", how="left")


merge bb_final → bdf (bureau.csv)

In [35]:
bdf = bdf.merge(bb_final, on="SK_ID_BUREAU", how="left")


In [36]:
baldf.shape


(27299925, 11)

In [37]:
bdf.shape


(1716428, 26)

In [38]:
bdf = bdf.fillna(0)


One-hot encode categorical columns

In [39]:
obj_cols = bdf.select_dtypes(include='object').columns.tolist()
obj_cols


['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']

In [40]:
bdf = pd.get_dummies(bdf, columns=obj_cols, drop_first=True)


Build the aggregation dictionary

In [41]:
num_cols = bdf.select_dtypes(include=['int64','float64']).columns.tolist()
num_cols.remove("SK_ID_CURR")
num_cols.remove("SK_ID_BUREAU")


In [42]:
agg_dict = {}

for col in num_cols:
    agg_dict[col] = ['mean','max','min','sum']


Aggregate per customer

In [43]:
bdf_agg = bdf.groupby("SK_ID_CURR").agg(agg_dict)


In [44]:
bdf_agg.columns = ["BUREAU_" + "_".join(col).upper() for col in bdf_agg.columns]
bdf_agg = bdf_agg.reset_index()


Merge aggregated bureau into main training data

In [48]:
mtraindf = joblib.load("clean_train.pkl")
mtestdf = joblib.load("clean_test.pkl")


In [50]:
mtraindf = mtraindf.merge(bdf_agg, on="SK_ID_CURR", how="left")
mtraindf = mtraindf.fillna(0)


In [51]:
mtestdf = mtestdf.merge(bdf_agg, on="SK_ID_CURR", how="left")
mtestdf = mtestdf.fillna(0)


In [53]:
mtestdf.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_BB_STATUS_5_MIN,BUREAU_BB_STATUS_5_SUM,BUREAU_BB_STATUS_C_MEAN,BUREAU_BB_STATUS_C_MAX,BUREAU_BB_STATUS_C_MIN,BUREAU_BB_STATUS_C_SUM,BUREAU_BB_STATUS_X_MEAN,BUREAU_BB_STATUS_X_MAX,BUREAU_BB_STATUS_X_MIN,BUREAU_BB_STATUS_X_SUM
0,100001,0.0,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329.0,...,0.0,0.0,15.714286,44.0,0.0,110.0,4.285714,9.0,0.0,30.0
1,100005,0.0,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469.0,...,0.0,0.0,1.666667,5.0,0.0,5.0,0.666667,1.0,0.0,2.0
2,100013,0.0,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458.0,...,0.0,0.0,25.750000,44.0,0.0,103.0,10.250000,40.0,0.0,41.0
3,100028,0.0,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866.0,...,0.0,0.0,21.916667,62.0,0.0,263.0,11.083333,60.0,0.0,133.0
4,100038,0.0,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [54]:


joblib.dump(mtraindf, "train_with_bureau.pkl")
joblib.dump(mtestdf, "test_with_bureau.pkl")


['test_with_bureau.pkl']